#**Duplicate Text Predictor**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np 
import pandas as pd 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model


# Read the train and test dataset

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/DSC/NLP_Workshop/train.csv')

In [ ]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [ ]:
test_data = pd.read_csv('/content/drive/MyDrive/DSC/NLP_Workshop/test.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
test_data.head()

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


In [ ]:
X_train = df.iloc[:,:5].values
Y_train = df.iloc[:,5:].values

In [ ]:
X_testq1 = test_data.iloc[:400001,1:2].values
X_testq2 = test_data.iloc[:400001, 2:].values

In [ ]:
Testid=test_data.iloc[:400001, 0:1].values

In [ ]:
s1 = X_train[:,3]
s2 = X_train[:,4]

# Text Pre-processing


## Tokenization



In [ ]:
def tokenize(s):
    tokens = []
    tokens = [word_tokenize(str(sentence)) for sentence in s]

    rm1 = []
    for w in tokens:
        sm = re.sub('[^A-Za-z]',' ', str(w))
        x = re.split("\s", sm)
        rm1.append(x)
        
    return rm1

## Lowercasing


In [ ]:
def lower_case(s):
    #Removing whitespaces    
    for sent in s:
        while '' in sent:
            sent.remove('')

    # Lowercasing
    low = []
    for i in s:
        i = [x.lower() for x in i]
        low.append(i)
        
    return low
    

## Normalization



In [ ]:
def lemmatize(s):
    lemma = []
    wnl = WordNetLemmatizer()
    for doc in s:
        tokens = [wnl.lemmatize(w) for w in doc]
        lemma.append(tokens)

    # Removing Stopwords
    filter_words = []
    Stopwords = set(stopwords.words('english'))

    #ab = spell('nd')
    for sent in lemma:
        tokens = [w for w in sent if w not in Stopwords]
        filter_words.append(tokens)

    space = ' ' 
    sentences = []
    for sentence in filter_words:
        sentences.append(space.join(sentence))
        
    return sentences

In [ ]:
# sent1 = tokenize(s1)
# sent2 = tokenize(s2)
# q1 = lower_case(sent1)
# q2 = lower_case(sent2)
# listq1 = lemmatize(q1)
# listq2 = lemmatize(q2)
# sent1_t = tokenize(X_test_q1)
# sent2_t = tokenize(X_test_q2)
# q1_t = lower_case(sent1_t)
# q2_t = lower_case(sent2_t)
# listq1 = lemmatize(q1_t)
# listq2 = lemmatize(q2_t)

When approaching a NLP problem, either you can perform all the above mentioned steps in that order or you can use Keras' Tokenizer class to perform tokenization.

# Keras text preprocessing



In [ ]:
MAX_NB_WORDS = 200000
tokenizer = Tokenizer(num_words = MAX_NB_WORDS)
tokenizer.fit_on_texts(list(df['question1'].values.astype(str))+list(df['question2'].values.astype(str)))


## Padding and sequencing


In [ ]:
# X_train_q1 = tokenizer.texts_to_sequences(np.array(listq1))
X_train_q1 = tokenizer.texts_to_sequences(df['question1'].values.astype(str))
X_train_q1 = pad_sequences(X_train_q1, maxlen = 30, padding='post')

# X_train_q2 = tokenizer.texts_to_sequences(np.array(listq2))
X_train_q2 = tokenizer.texts_to_sequences(df['question2'].values.astype(str))
X_train_q2 = pad_sequences(X_train_q2, maxlen = 30, padding='post')


In [ ]:
X_test_q1 = tokenizer.texts_to_sequences(X_testq1.ravel())
X_test_q1 = pad_sequences(X_test_q1,maxlen = 30, padding='post')

X_test_q2 = tokenizer.texts_to_sequences(X_testq2.astype(str).ravel())
X_test_q2 = pad_sequences(X_test_q2, maxlen = 30, padding='post')

In [ ]:
word_index = tokenizer.word_index

## Loading Glove word embedding

In [ ]:
embedding_index = {}
#Glove embeddings with 6 Billion words with each word being a 200 dimensional vector.
with open('/content/drive/MyDrive/DSC/NLP_Workshop/glove.6B.200d.txt','r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], 'float32')
        embedding_index[word] = vectors
    f.close()

In [ ]:
embedding_matrix = np.random.random((len(word_index)+1, 200))
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# LSTM

In [ ]:
# Model for Q1
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization
model_q1 = tf.keras.Sequential()
model_q1.add(Embedding(input_dim = len(word_index)+1,
                       output_dim = 200,
                      weights = [embedding_matrix],
                      input_length = 30))
model_q1.add(LSTM(128, activation = 'tanh', return_sequences = True))
model_q1.add(Dropout(0.2))
model_q1.add(LSTM(128, return_sequences = True))
model_q1.add(LSTM(128))
model_q1.add(Dense(2, activation = 'sigmoid'))

In [ ]:
# Model for Q2
model_q2 = tf.keras.Sequential()
model_q2.add(Embedding(input_dim = len(word_index)+1,
                       output_dim = 200,
                      weights = [embedding_matrix],
                      input_length = 30))
model_q2.add(LSTM(128, activation = 'tanh', return_sequences = True))
model_q2.add(Dropout(0.2))
model_q2.add(LSTM(128, return_sequences = True))
model_q2.add(LSTM(128))
model_q2.add(Dense(60, activation = 'tanh'))
model_q2.add(Dense(2, activation = 'sigmoid'))

In [ ]:
# Merging the output of the two models,i.e, model_q1 and model_q2
mergedOut = Multiply()([model_q1.output, model_q2.output])

mergedOut = Flatten()(mergedOut)
mergedOut = Dense(100, activation = 'relu')(mergedOut)
mergedOut = Dropout(0.2)(mergedOut)
mergedOut = Dense(50, activation = 'relu')(mergedOut)
mergedOut = Dropout(0.2)(mergedOut)
mergedOut = Dense(2, activation = 'sigmoid')(mergedOut)

In [ ]:
new_model = Model([model_q1.input, model_q2.input], mergedOut)
new_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',
                 metrics = ['accuracy'])
history = new_model.fit([X_train_q1,X_train_q2],Y_train, batch_size = 2000, epochs = 10)

Epoch 1/10
203/203 [==============================] - 79s 341ms/step - loss: 0.5362 - accuracy: 0.7346
Epoch 2/10
203/203 [==============================] - 69s 341ms/step - loss: 0.5053 - accuracy: 0.7530
Epoch 3/10
203/203 [==============================] - 69s 342ms/step - loss: 0.4733 - accuracy: 0.7720
Epoch 4/10
203/203 [==============================] - 69s 340ms/step - loss: 0.4384 - accuracy: 0.7931
Epoch 5/10
203/203 [==============================] - 69s 340ms/step - loss: 0.4051 - accuracy: 0.8121
Epoch 6/10
203/203 [==============================] - 69s 338ms/step - loss: 0.3707 - accuracy: 0.8323
Epoch 7/10
203/203 [==============================] - 69s 340ms/step - loss: 0.3402 - accuracy: 0.8479
Epoch 8/10
203/203 [==============================] - 69s 340ms/step - loss: 0.3140 - accuracy: 0.8609
Epoch 9/10
203/203 [==============================] - 69s 341ms/step - loss: 0.2889 - accuracy: 0.8726
Epoch 10/10
203/203 [==============================] - 69s 341ms/step - l

In [ ]:
y_pred = new_model.predict([X_test_q1, X_test_q2], batch_size=2000, verbose=1)
y_pred += new_model.predict([X_test_q1, X_test_q2], batch_size=2000, verbose=1)
y_pred /= 2
result=(y_pred[:,0]+y_pred[:,1])/2

201/201 [==============================] - 20s 98ms/step


In [ ]:
res= pd.DataFrame({'Test_id':Testid, 'Predictions':result})

In [ ]:
res.to_csv('my_submission.csv', index=False)
print("Submission saved")

Submission saved
